In [ ]:
import yaml
from warnings import filterwarnings
from dotenv import load_dotenv

setting = yaml.safe_load(open("../setting.yml"))
print(setting)

filterwarnings("ignore")
load_dotenv()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-2-7b-chat-hf"
torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.float16,trust_remote_code=True,token =setting["hf_token"])

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,token =setting["hf_token"])


In [ ]:
from peft import PeftModel, PeftConfig

#Load the model weights from hub
model_adapters = "praveeny/phi2-webglm-qlora"
model = PeftModel.from_pretrained(model, model_adapters)

model = model.merge_and_unload()
model.save_pretrained("updated_adapters")

In [ ]:
model.push_to_hub("attack-llama-2-7b-chat-lora",
                  commit_message="merged model",token =setting["hf_token"])

tokenizer.push_to_hub("attack-llama-2-7b-chat-lora",
                  commit_message="tokenizer",token =setting["hf_token"])

In [ ]:
## Download the model from huggingface hub
from huggingface_hub import snapshot_download

#Download the merged model
model_id="Xcvddax/attack-llama-2-7b-chat-lora"
#Download the repository to local_dir
snapshot_download(repo_id=model_id, local_dir="model",
                  local_dir_use_symlinks=False)